In [1]:
import pickle
import numpy as np
import pandas as pd

from datetime import datetime as dt
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from category_encoders.one_hot import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

Сохраняем необходимые данные (чтобы не качать при каждом перезапуске ноутбука).

In [2]:
# cкачиваем все данные из post_text
post_text = pd.read_sql(
    "SELECT * FROM public.post_text_df",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем post_text в файл
post_text.to_csv('post_text.csv', index=False)

In [3]:
# скачиваем среднее по target по каждому post_id из feed_data
post_stat = pd.read_sql(
    "SELECT post_id, AVG(target) AS post_stat FROM public.feed_data GROUP BY post_id",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем post_text в файл
post_stat.to_csv('post_stat.csv', index=False)

In [4]:
# cкачиваем все данные из user_data
user_data = pd.read_sql(
    "SELECT * FROM public.user_data",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем user_data в файл
user_data.to_csv('user_data.csv', index=False)

In [5]:
# скачиваем среднее по target по каждому user_id из feed_data
user_stat = pd.read_sql(
    "SELECT user_id, AVG(target) AS user_stat FROM public.feed_data GROUP BY user_id",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем user_data в файл
user_stat.to_csv('user_stat.csv', index=False)

In [6]:
# скачиваем последние 100 тысяч записей в feed_data
feed_data = pd.read_sql(
    "SELECT * FROM public.feed_data WHERE action = 'view' ORDER BY timestamp DESC LIMIT 100000",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
# записываем feed_data в файл
feed_data.to_csv('feed_data.csv', index=False)

Загружаем данные из ранее сохранённых файлов

In [7]:
post_text = pd.read_csv('post_text.csv')

In [8]:
post_stat = pd.read_csv('post_stat.csv')

In [9]:
user_data = pd.read_csv('user_data.csv')

In [10]:
user_stat = pd.read_csv('user_stat.csv')

In [11]:
feed_data = pd.read_csv('feed_data.csv')
feed_data.sort_values('timestamp', inplace=True, ignore_index=True)

В таблице feed_data есть колонка 'target', она будет являться целевой в нашем обучении.

0 - пост не получил like от пользователя, которому был показан.

1 - пост получил like.

При помощи метода TF-IDF в таблице post_text преобразуем текстовую колонку 'text' в векторный вид.
Из получившейся матрицы возьмём построчные максимум.

In [12]:
post_text_copy = post_text.copy()

vectorizer = TfidfVectorizer()
F = vectorizer.fit_transform(post_text['text'])
post_text_copy['text'] = F.toarray().max(axis=1)

Соединяем таблицы по колонкам post_id и user_id.

In [13]:
df = pd.merge(feed_data,
              post_text_copy,
              on='post_id',
              how='left')

In [14]:
df = pd.merge(df,
              post_stat,
              on='post_id',
              how='left')

In [15]:
df = pd.merge(df,
              user_data,
              on='user_id',
              how='left')

In [16]:
df = pd.merge(df,
              user_stat,
              on='user_id',
              how='left')

Из колонки timestamp выделяем дополнительные признаки: год, месяц, день, час, минута, секунда. А исходную колонку удаляем.

In [17]:
df['timestamp'] = df['timestamp'].astype('datetime64')

df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
df['second'] = df['timestamp'].dt.second

df = df.drop('timestamp', axis=1)

Делим сгруппированый набор данных на признаки "X" и целевую колонку "Y".

In [18]:
X = df.drop(['user_id', 'post_id', 'action', 'target'], axis=1)
y = df['target']

Делим колонки в таблице признаков на числовые и категориальные, последние в свою очередь делим на признаки, которые будут обрабатываться методом One-Hot или кодирования по таргету.

In [19]:
numeric_columns = list(X.select_dtypes(exclude='object').columns)
categorical_columns = list(X.select_dtypes(include='object').columns)

columns_ohe = [x for x in categorical_columns if X[x].nunique() < 5]
columns_mte = [x for x in categorical_columns if X[x].nunique() >= 5]

numeric_columns_ind = [list(X.columns).index(col) for col in numeric_columns]
columns_ohe_ind = [list(X.columns).index(col) for col in columns_ohe]
columns_mte_ind = [list(X.columns).index(col) for col in columns_mte]

Создаём трансформер, который будет преобразовывать данные признаков следующим образом: числовые признаки стандартизирует, а к категориальным применит метод One-Hot и TargetEncoder.

In [20]:
t = [('StandardScaler', StandardScaler(), numeric_columns_ind),
     ('OneHotEncoder', OneHotEncoder(), columns_ohe_ind),
     ('MeanTargetEncoder', TargetEncoder(), columns_mte_ind)]

columns_transformer = ColumnTransformer(transformers=t)

D:\anaconda3\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
D:\anaconda3\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


Делим данные на тренеровочные и тестовые.

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=1)

Проверяем баланс классов на тренеровке и на тесте.

In [22]:
y_train.value_counts()[1] / len(y_train)

0.1572125

In [23]:
y_test.value_counts()[1] / len(y_test)

0.1559

Стоит отметить, что в данных, на которых выполняется обучение, целевые классы имеют явный дисбаланс: класс 1 порядка 16%. При разбивке данных на тренеровочные и тестывые соотношение классов будем считать приемлемым.

При помощи Pipeline проводим обучение модели CatBoostClassifier. Параметры модели были подобранны вручную.

In [24]:
%%time

pipe = Pipeline([('columns_transformer', columns_transformer),
                 ('decision_tree', CatBoostClassifier(iterations = 10,
                                                      loss_function = 'MultiClass',
                                                      early_stopping_rounds=10,
                                                      thread_count = 100,
                                                      random_state=1))])
pipe.fit(X_train, y_train)

D:\anaconda3\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
D:\anaconda3\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


Learning rate set to 0.5
0:	learn: 0.5100789	total: 201ms	remaining: 1.8s
1:	learn: 0.4520958	total: 210ms	remaining: 840ms
2:	learn: 0.4279819	total: 223ms	remaining: 521ms
3:	learn: 0.4184583	total: 236ms	remaining: 354ms
4:	learn: 0.4145570	total: 248ms	remaining: 248ms
5:	learn: 0.4114360	total: 258ms	remaining: 172ms
6:	learn: 0.4101998	total: 269ms	remaining: 115ms
7:	learn: 0.4072614	total: 280ms	remaining: 70ms
8:	learn: 0.4054249	total: 291ms	remaining: 32.3ms
9:	learn: 0.4048414	total: 302ms	remaining: 0us
CPU times: total: 1.72 s
Wall time: 1.4 s


Pipeline(steps=[('columns_transformer',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  [0, 2, 3, 4, 7, 10, 11, 12,
                                                   13, 14, 15, 16]),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(), [8, 9]),
                                                 ('MeanTargetEncoder',
                                                  TargetEncoder(),
                                                  [1, 5, 6])])),
                ('decision_tree',
                 <catboost.core.CatBoostClassifier object at 0x00000260EEBFCF10>)])

Сохраняем обученую модель.

In [25]:
filename = 'model_control.pkl'
pickle.dump(pipe, open(filename, 'wb'))

Рассчитаем ROC-AUC для обученной модели на тестовых данных.

In [26]:
print(f'ROC-AUC {roc_auc_score(y_test, pipe.predict_proba(X_test)[:, 1])}')

ROC-AUC 0.6493664586068838


# Строим таблицы для рекомендации постов

In [27]:
note = post_text.copy()

vectorizer = TfidfVectorizer()
F = vectorizer.fit_transform(note['text'])
note['text'] = F.toarray().max(axis=1)

In [28]:
note = pd.merge(note,
                post_stat,
                on='post_id',
                how='left')

Сохраняем таблицу note на сервер и сразу скачиваем её для проверки, что всё сохранилось как и задумывалось.

In [29]:
note.to_sql(
    "note_control_shaverdin",
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml",
    schema="public",
    if_exists='replace',
    index=False
)

note_shaverdin = pd.read_sql(
    """SELECT * FROM note_control_shaverdin""",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
note_shaverdin

,post_id,text,topic,post_stat
0,1,0.439495,business,0.111436
1,2,0.290946,business,0.078333
2,3,0.279045,business,0.117684
3,4,0.525321,business,0.125523
4,5,0.409826,business,0.118426
...,...,...,...,...
7018,7315,0.226524,movie,0.133903
7019,7316,0.333130,movie,0.093392
7020,7317,0.507582,movie,0.097027
7021,7318,0.263741,movie,0.091092


In [30]:
users = pd.merge(user_data,
                 user_stat,
                 on='user_id',
                 how='left')

Сохраняем таблицу users на сервер и сразу скачиваем её для проверки, что всё сохранилось как и задумывалось.

In [31]:
users.to_sql(
    "users_shaverdin",
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml",
    schema="public",
    if_exists='replace',
    index=False
)

users_shaverdin = pd.read_sql(
    """SELECT * FROM users_shaverdin""",
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@""postgres.lab.karpov.courses:6432/startml"
)
users_shaverdin

,user_id,gender,age,country,city,exp_group,os,source,user_stat
0,200,1,34,Russia,Degtyarsk,3,Android,ads,0.107232
1,201,0,37,Russia,Abakan,0,Android,ads,0.077540
2,202,1,17,Russia,Smolensk,4,Android,ads,0.120166
3,203,0,18,Russia,Moscow,1,iOS,ads,0.159686
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads,0.142857
...,...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic,0.052356
163201,168549,0,18,Russia,Tula,2,Android,organic,0.080292
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic,0.093366
163203,168551,0,38,Russia,Moscow,3,iOS,organic,0.091429
